#### Import libs

In [1]:
#base
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as numpy
#torch
import torch
from torch import nn

import warnings
warnings.filterwarnings('ignore')

#transformers
import transformers

> Read Json Lines file

In [2]:
df = pd.read_json("/home/marena/Elbrus_phase_2/nlp_project/data/kinopoisk.jsonl", lines=True)

In [3]:
df

,part,movie_name,review_id,author,date,title,grade3,grade10,content
0,top250,Блеф (1976),17144,Come Back,2011-09-24,Плакали наши денежки ©,Good,10.0,"\n""Блеф» — одна из моих самых любимых комедий...."
1,top250,Блеф (1976),17139,Stasiki,2008-03-04,None,Good,0.0,\nАдриано Челентано продолжает радовать нас св...
2,top250,Блеф (1976),17137,Flashman,2007-03-04,None,Good,10.0,"\nНесомненно, это один из великих фильмов 80-х..."
3,top250,Блеф (1976),17135,Sergio Tishin,2009-08-17,""" Черное, красное, ерунда это все. Выигрывает ...",Good,0.0,\nЭта фраза на мой взгляд отражает сюжет несом...
4,top250,Блеф (1976),17151,Фюльгья,2009-08-20,"«Он хотел убежать? Да! Блеф, блеф…»",Neutral,7.0,"\n- как пела Земфира, скорее всего, по соверше..."
...,...,...,...,...,...,...,...,...,...
36586,bottom100,Цветок дьявола (2010),25123,bestiya163,2010-09-23,"Ой, ой, ой!",Bad,2.0,\n Ну с чего бы начать… Давненько я не пи...
36587,bottom100,Цветок дьявола (2010),25192,Молка,2010-10-02,Молчаливый мужик на коне…,Bad,1.0,"\n Можно начать с того, что уже постер к ..."
36588,bottom100,Цветок дьявола (2010),25080,jetry,2010-09-16,Это проявилось сегодня ночью.,Good,7.0,"\n Фильм производства России, поэтому мно..."
36589,bottom100,Цветок дьявола (2010),25088,Alkort,2010-09-16,«Finita la comedia»,Bad,0.0,\n 16 сентября на большие экраны вышел «м...


In [4]:
df['grade3'].value_counts()

grade3
Good       27264
Bad         4751
Neutral     4576
Name: count, dtype: int64

> Выделим важные данные для обучения в отдельные датасеты

In [5]:
x = df['content']
y = df['grade3']

1. Обучение моделей. Классический ML. BoW/TF-IDF + ML-алгосы

Для того, чтобы подать текст в TF-IDF, BoW и далее в ML алгоритм, необходимо его отчистить и провести либо Лемматизацию, либо Стемминг. Выберем Лемматизацию, т.к. она точнее 

In [6]:
import re
import nltk
import string
import pymorphy2
from nltk.corpus import stopwords

nltk.download('stopwords')

# Инициализируем лемматизатор и список стоп-слов
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def clean_text(text):
    if not isinstance(text, str):  # Проверяем, что текст строка
        return ""

    text = text.lower()  # Приводим к нижнему регистру
    text = text.translate(str.maketrans('', '', string.punctuation))  # Удаляем пунктуацию
    tokens = text.split()  # Разбиваем на слова
    tokens = [morph.parse(word)[0].normal_form for word in tokens if word not in stop_words]  # Лемматизация
    tokens = [word for word in tokens if len(word) > 1]  # Убираем слишком короткие слова
    cleaned_text = " ".join(tokens).strip()  # Убираем лишние пробелы
    return cleaned_text if cleaned_text else None 

[nltk_data] Downloading package stopwords to /home/marena/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Тестируем

In [7]:
clean_text(x[0])

'блеф» один мой самый любимый комедия фильм наверно смотреть сто блефовать видеть мочь выразить свой восхищение главный действовать лицо фильм начать адриано челентано который считать это хороший роль кино великолепный актёр неплохой певец странно родина италия песня мало слушать думать итальянец француз привыкнуть тот сей актёр популярный свой родина парадокс челентано профессионал свой дело комик серьёзный выражение лицо смешной ещё одновременно серъёзный адриано браво несколько слово энтони куиня самый горбун нотрдама собор парижский богоматерь оригинальный версия смотреть рекомендовать както приключиться один интересный история съёмка один свой фильм сломать подвихнуть нога роль требовать один сцена кружиться дама танец вместе съёмочный коллектив выйти положение сделать вращаться платформа который создавать видимость весь ритм танец такой история слово замечательный актёр фильм жанр который смело обозвать авантюра комедийный авантюра некий белля дьюк весьма влиятельный дама пытатьс

Работает, идём дальше. Теперь почистим весь наш датасет

In [8]:
#x = x.apply(clean_text)

In [9]:
# Создадим и сохраним новый очищенный датасет для удобства работы (и чтобы не ждать по 30 минут, пока пройдёт лемматизация и очистка текста)
save = pd.DataFrame({'Content': x, 'Target': y})

save.to_csv('save.csv')

Разделим на 2 основных метода TF-IDF и BoW(Bag of Words) и попробуем прогнать через модели CatBoost, XGBoost и LogReg

In [10]:
save = pd.read_csv('save.csv')

In [11]:
x = save['Content']
y = save['Target']

In [12]:
#Imports
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [13]:
#TF-IDF
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
tfidf_text = tfidf.fit_transform(x)

In [14]:
#BoW
BoW = CountVectorizer(max_features=20000, ngram_range=(1,2), min_df=5, max_df=0.9)
bow_text = BoW.fit_transform(x)

Закодируем таргет, чтобы обучить модели

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [16]:
encoder = LabelEncoder()
target_ml = encoder.fit_transform(y)

Делаем разделение

In [17]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(tfidf_text, target_ml, test_size=0.2, random_state=42)
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow_text, target_ml, test_size=0.2, random_state=42)

Обучаем CatBoost, LogReg и XGBoost

In [18]:
#learning rate
lr = 0.01

In [19]:
#CatBoost
from catboost import CatBoostClassifier

model_cb = CatBoostClassifier(
    iterations=1000,
    learning_rate=lr,
    depth=6,
    l2_leaf_reg=3,
    loss_function='MultiClass',
    auto_class_weights='Balanced',
    task_type='GPU'
)

In [20]:
model_cb.fit(X_train_tf, y_train_tf)

0:	learn: 1.0936410	total: 621ms	remaining: 10m 20s
1:	learn: 1.0888047	total: 1.13s	remaining: 9m 21s
2:	learn: 1.0840580	total: 1.63s	remaining: 9m 1s
3:	learn: 1.0794321	total: 2.14s	remaining: 8m 53s
4:	learn: 1.0750258	total: 2.66s	remaining: 8m 50s
5:	learn: 1.0706879	total: 3.18s	remaining: 8m 46s
6:	learn: 1.0664126	total: 3.7s	remaining: 8m 44s
7:	learn: 1.0622102	total: 4.21s	remaining: 8m 42s
8:	learn: 1.0581018	total: 4.7s	remaining: 8m 37s
9:	learn: 1.0541058	total: 5.18s	remaining: 8m 33s
10:	learn: 1.0501841	total: 5.66s	remaining: 8m 28s
11:	learn: 1.0463964	total: 6.12s	remaining: 8m 24s
12:	learn: 1.0426260	total: 6.59s	remaining: 8m 20s
13:	learn: 1.0389600	total: 7.1s	remaining: 8m 20s
14:	learn: 1.0354199	total: 7.58s	remaining: 8m 17s
15:	learn: 1.0319847	total: 8.08s	remaining: 8m 16s
16:	learn: 1.0285302	total: 8.6s	remaining: 8m 17s
17:	learn: 1.0251966	total: 9.12s	remaining: 8m 17s
18:	learn: 1.0219064	total: 9.6s	remaining: 8m 15s
19:	learn: 1.0186603	total:

In [21]:
#LogReg
from sklearn.linear_model import LogisticRegression

model_logreg = LogisticRegression(C=1.0, penalty='l2', class_weight='balanced', solver='lbfgs')

#
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators=1000, learning_rate=0.01, max_depth=6, class_weight='balanced', tree_method='gpu_hist')

In [22]:
model_logreg.fit(X_train_tf, y_train_tf)

LogisticRegression(class_weight='balanced')

In [23]:
model_xgb.fit(X_train_tf, y_train_tf)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, ...)

In [24]:
from sklearn.metrics import f1_score

y_pred_xgb = model_xgb.predict(X_test_tf)
print("XGBoost F1-score:", f1_score(y_test_tf, y_pred_xgb, average='weighted'))

y_pred_logreg = model_logreg.predict(X_test_tf)
print("LogReg F1-score:", f1_score(y_test_tf, y_pred_logreg, average='weighted'))

y_pred_cat = model_cb.predict(X_test_tf)
print("CatBoost F1-score:", f1_score(y_test_tf, y_pred_cat, average='weighted'))

XGBoost F1-score: 0.6288034811030463
LogReg F1-score: 0.7980095266941593
CatBoost F1-score: 0.6651485345040407


Выгрузим веса на всякий случай

In [26]:
import joblib
joblib.dump(model_xgb, "/home/marena/Elbrus_phase_2/nlp_project/models/model_1/xgboost_model.pkl")
joblib.dump(model_logreg, "/home/marena/Elbrus_phase_2/nlp_project/models/model_1/logreg_model.pkl")
model_cb.save_model("/home/marena/Elbrus_phase_2/nlp_project/models/model_1/catboost_model.cbm")

In [25]:
#Это был TF-IDF
#Теперь попробуем BoW 

In [27]:
model_cb_2 = CatBoostClassifier(
    iterations=1000,
    learning_rate=lr,
    depth=6,
    l2_leaf_reg=3,
    loss_function='MultiClass',
    auto_class_weights='Balanced',
    task_type='GPU'
)

model_logreg_2 = LogisticRegression(C=1.0, penalty='l2', class_weight='balanced', solver='lbfgs')

model_xgb_2 = XGBClassifier(n_estimators=1000, learning_rate=0.01, max_depth=6, class_weight='balanced', tree_method='gpu_hist')

In [28]:
model_cb_2.fit(X_train_bow, y_train_bow)

0:	learn: 1.0936578	total: 118ms	remaining: 1m 57s
1:	learn: 1.0888440	total: 223ms	remaining: 1m 51s
2:	learn: 1.0841535	total: 317ms	remaining: 1m 45s
3:	learn: 1.0795719	total: 407ms	remaining: 1m 41s
4:	learn: 1.0751309	total: 503ms	remaining: 1m 40s
5:	learn: 1.0707907	total: 590ms	remaining: 1m 37s
6:	learn: 1.0665209	total: 676ms	remaining: 1m 35s
7:	learn: 1.0623402	total: 758ms	remaining: 1m 33s
8:	learn: 1.0582547	total: 844ms	remaining: 1m 32s
9:	learn: 1.0542518	total: 931ms	remaining: 1m 32s
10:	learn: 1.0503458	total: 1.01s	remaining: 1m 31s
11:	learn: 1.0465560	total: 1.09s	remaining: 1m 30s
12:	learn: 1.0428046	total: 1.17s	remaining: 1m 29s
13:	learn: 1.0391561	total: 1.26s	remaining: 1m 28s
14:	learn: 1.0356152	total: 1.35s	remaining: 1m 28s
15:	learn: 1.0321433	total: 1.44s	remaining: 1m 28s
16:	learn: 1.0287188	total: 1.52s	remaining: 1m 28s
17:	learn: 1.0253629	total: 1.61s	remaining: 1m 28s
18:	learn: 1.0220945	total: 1.7s	remaining: 1m 27s
19:	learn: 1.0188566	to

In [29]:
model_logreg_2.fit(X_train_bow, y_train_bow)

LogisticRegression(class_weight='balanced')

In [30]:
model_xgb_2.fit(X_train_bow, y_train_bow)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, ...)

In [33]:
y_pred_xgb_bow = model_xgb_2.predict(X_test_bow)
print("XGBoost F1-score:", f1_score(y_test_bow, y_pred_xgb_bow, average='weighted'))

y_pred_logreg_bow = model_logreg_2.predict(X_test_bow)
print("LogReg F1-score:", f1_score(y_test_bow, y_pred_logreg_bow, average='weighted'))

y_pred_cat_bow = model_cb_2.predict(X_test_bow)
print("CatBoost F1-score:", f1_score(y_test_bow, y_pred_cat_bow, average='weighted'))

XGBoost F1-score: 0.6285488194841349
LogReg F1-score: 0.7922777139811713
CatBoost F1-score: 0.6716834192328344
